# NBA DRAFT

Testing if Git server extension works

# Imports

In [1]:
import pandas as pd

In [ ]:
df = pd.read_csv("/data/2024_NBA

# References

Basketball Reference

Sports Reference

NBADraft.net